# Constrained optimization: Penalty Method
* ## Giacomo Bacchetta, bacchetta.1840949@studenti.uniroma1.it
* ## Edoardo Cesaroni, cesaroni.1841742@studenti.uniroma1.it 
* ## Fabio Ciccarelli, ciccarelli.1835348@studenti.uniroma1.it 

## Packages' import

We import the python libraries useful for the implementation of the method. These are:
- $\bf{Autograd.numpy}$, which allows us to use siple mathematical functions;
- $\bf{Autograd}$, used to calculate the gradient (*grad*) of several functions;
- $\bf{AIL}$, which contains the Truncated Newthon Method as unconstrained opitimization method applied below.

In [1]:
import autograd.numpy as np
from autograd import grad
from AIL import nt

## KKT's Conditions

This function allows to verify with an accuracy equal to Epsilon how much a point respects the Karush-Kuhn-Tucker conditions that are the necessary conditions of first order for a constrained optimization problem.

In [2]:
def verify_KKT(f, g, h, x, lam, mu):
    epsilon = 1e-3
    
    for i in range(len(g)):
        if g[i](x) > epsilon or lam[i]*g[i](x) > epsilon or lam[i]*g[i](x) < -epsilon:
            return False
        
    for j in range(len(h)):
        if h[j](x) > epsilon or h[j](x) < -epsilon:
            return False

    if np.linalg.norm(grad(f)(x) + sum(grad(g[i])(x)*lam[i] for i in range(len(g))) + sum(grad(h[j])(x)*mu[j] for j in range(len(h)))) > epsilon:
        return False

    return True

## Main

Before moving to the core of the algorithm, we defined the P function that replicates the quadratic penalty function.

In [3]:
def P(x):
    
    return f(x)+ (1/epsilon_p)*sum(max(0, g[i](x))**2 for i in range(len(g)))+ (1/epsilon_p)*sum(h[j](x)**2 for j in range(len(h)))

In [4]:
def penalty_method(f, g, h, x):
    
    global epsilon_p

    theta1 = 0.5
    theta2 = 0.5
    theta3 = 0.5
    delta = 10
    epsilon_prev = 0.1                 
    epsilon_current = 0.1
    epsilon_p = epsilon_current
    ite = 0
    
    
    while True:
        
        lam = []
        mu = []

        for i in range(len(g)):
            lam.append((max(0, g[i](x)))*(2/epsilon_prev))

        for j in range(len(h)):
            mu.append((h[j](x))*(2/epsilon_prev))

        verifica_KKT = verify_KKT(f, g, h, x, lam, mu)
        if verify_KKT(f, g, h, x, lam, mu) == True:
            print('*'*70)
            print('PM: STOP! KKT condition is verified.')
            print('PM: The optimum point is:',x, 'with lambda=', lam, 'and mu=', mu)
            print('PM: The optimum value of the bbjective function is:', f(x))
            return x

        epsilon_p = epsilon_current

        x_succ = nt(P, x, delta)
    

        if sum(max(0, g[i](x_succ))**2 for i in range(len(g))) + sum((h[j](x_succ))**2 for j in range(len(h))) <= theta1*(sum(max(0, g[i](x))**2 for i in range(len(g))) + sum((h[j](x))**2 for j in range(len(h)))):
            epsilon_prev = epsilon_current
            epsilon_current = epsilon_current
            
        elif epsilon_current >= 1e-6: 
            epsilon_prev = epsilon_current
            epsilon_current = theta2 * epsilon_current
            
        elif epsilon_current < 1e-6:
            print('*'*70)
            print('PM: STOP! Epsilon too little, the best point is:', x_succ)
            print('PM: The best value of the objective function is:', f(x_succ))
            return x_succ

        if delta > 1e-4:
            delta = delta * theta3
        x = x_succ
        
        ite = ite + 1

## IMPLEMENTATION TEST

### Beale's Problem

In [5]:
g = []
h = []

def f(x):
    return 9-8*x[0]-6*x[1]-4*x[2]+2*x[0]**2+2*x[1]**2+x[2]**2+2*x[0]*x[1]+2*x[0]*x[2]

def g1(x):
    return -(3-x[0]-x[1]-2*x[2]) 
g.append(g1)

def g2(x):
    return -(x[0]) 
g.append(g2)

def g3(x):
    return -(x[1]) 
g.append(g3)

def g4(x):
    return -(x[2]) 
g.append(g4)

x0 = np.array([0.5, 0.5, 0.5])


penalty_method(f, g, h, x0);

**********************************************************************
PM: STOP! KKT condition is verified.
PM: The optimum point is: [1.33321742 0.77785486 0.44463741] with lambda= [0.22214344452684998, 0.0, 0.0, 0.0] and mu= []
PM: The optimum value of the bbjective function is: 0.11103399135715675


### HS 108

In [6]:
g = []
h = []

def f(x):
    return -0.5*(x[0]*x[3]-x[1]*x[2]+x[2]*x[8]-x[4]*x[8]+x[4]*x[7]-x[5]*x[6])

def g1(x):
    return -(1-x[2]**2-x[3]**2)
g.append(g1)

def g2(x):
    return -(1-x[4]**2-x[5]**2)
g.append(g2)

def g3(x):
    return -(1-(x[0]-x[4])**2-(x[1]-x[5])**2)
g.append(g3)

def g4(x):
    return -(1-(x[0]-x[6])**2-(x[1]-x[7])**2)
g.append(g4)

def g5(x):
    return -(1-(x[2]-x[4])**2-(x[3]-x[5])**2)
g.append(g5)

def g6(x):
    return -(1-(x[2]-x[6])**2-(x[3]-x[7])**2)
g.append(g6)

def g7d(x):
    return -(1-x[6]**2-(x[7]-x[8])**2)
g.append(g7d)

def g7s(x):
    return -(x[2]*x[8])
g.append(g7d)

def g8d(x):
    return -(x[4]*x[7]-x[5]*x[6])
g.append(g8d)

def g8s(x):
    return -(1-x[8]**2)
g.append(g8s)

def g9d(x):
    return -(1-x[0]**2-(x[1]-x[8])**2)
g.append(g9d)

def g9s(x):
    return -(x[0]*x[3]-x[1]*x[2])
g.append(g9s)

def g10d(x):
    return x[4]*x[8]
g.append(g10d)

def g10s(x):
    return -x[8]
g.append(g10s)

x0 = np.array([1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])


penalty_method(f, g, h, x0);

**********************************************************************
PM: STOP! KKT condition is verified.
PM: The optimum point is: [ 9.43168974e-01  3.32483147e-01  1.83597405e-01  9.83058880e-01
  9.43152661e-01  3.32529423e-01  1.83624450e-01  9.83025136e-01
 -1.48722151e-08] with lambda= [0.14434496036415112, 0.1443308943404098, 0.0, 0.14433777795005653, 0.14433394707388914, 0.0, 0.07217266267630862, 0.07217266267630862, 0.0, 0.0, 0.1443408850083472, 0.0, 0.0, 1.9036435334339376e-05] and mu= []
PM: The optimum value of the bbjective function is: -0.8661149218655723


### HS 71

In [7]:
g = []
h = []

def f(x):
    return x[0]*x[3]*(x[0]+x[1]+x[2])+x[2]

def g1(x):
    return -((x[0]*x[1]*x[2]*x[3]) - 25.)
g.append(g1)

def h1(x):
    return ((((x[0])**2)+((x[1])**2)+((x[2])**2)+((x[3])**2)) - 40.)
h.append(h1)

def g2d(x):
    return x[0] - 5
g.append(g2d)

def g2s(x):
    return -(x[0] - 1)
g.append(g2s)

def g3d(x):
    return x[1] - 5
g.append(g3d)

def g3s(x):
    return -(x[1] - 1)
g.append(g3s)

def g4d(x):
    return x[2] - 5
g.append(g4d)

def g4s(x):
    return -(x[2] - 1)
g.append(g4s)

def g5d(x):
    return x[3] - 5
g.append(g5d)

def g5s(x):
    return -(x[3] - 1)
g.append(g5s)

x0 = np.array([1., 5., 5., 1.])  


penalty_method(f, g, h, x0);

**********************************************************************
PM: STOP! KKT condition is verified.
PM: The optimum point is: [0.99915175 4.7430355  3.82090078 1.38063515] with lambda= [0.5523182383103631, 0.0, 1.0857659920208107, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] and mu= [0.1614833604253363]
PM: The optimum value of the bbjective function is: 17.012836625500288


### HS 76

In [8]:
g = []
h = []

def f(x):
    return x[0]**2 + 0.5*(x[1]**2) + x[2]**2 + 0.5*(x[3]**2) - (x[0]*x[2]) + (x[2]*x[3]) - x[0] - 3*x[1] + x[2] - x[3]

def g1(x):
    return -(5. - x[0] - 2*x[1] - x[2] - x[3])
g.append(g1)

def g2(x):
    return -(4. - 3.*x[0] - x[1] - 2.*x[2] + x[3])
g.append(g2)

def g3(x):
    return -(x[1] + 4. * x[2] - 1.5)
g.append(g3)

def g4(x):
    return -x[0]
g.append(g4)

def g5(x):
    return -x[1]
g.append(g5)

def g6(x):
    return -x[2]
g.append(g6)

def g7(x):
    return -x[3]
g.append(g7)

x0 = np.array([.5, .5, .5, .5])


penalty_method(f, g, h, x0);

**********************************************************************
PM: STOP! KKT condition is verified.
PM: The optimum point is: [ 2.72580331e-01  2.09100275e+00 -3.37320662e-04  5.45840261e-01] with lambda= [0.45449936106535915, 0.0, 0.0, 0.0, 0.0, 1.7270817910756555, 0.0] and mu= []
PM: The optimum value of the bbjective function is: -4.682441141853801


### HS 100

In [9]:
g = []
h = []

def f(x):
    return (x[0] - 10.)**2 + 5.*(x[1]-12.)**2 + x[2]**4 + 3*(x[3] - 11.)**2 + 10.*x[4]**6 + 7.*x[5]**2 + x[6]**4 - 4.*x[5]*x[6] - 10*x[5] - 8.*x[6]

def g1(x):
    return -(127 -2*x[0]**2 -3*x[1]**4 - x[2] - 4*x[3]**2 - 5*x[4])
g.append(g1)

def g2(x):
    return -(282 - 7*x[0] - 3*x[1] -10*x[2]**2 -x[3] + x[4])
g.append(g2)

def g3(x):
    return -(196 - 23*x[0] - x[1]**2 -6*x[5]**2+8*x[6])
g.append(g3)

def g4(x):
    return -(-4*x[0]**2 -x[1]**2 +3*x[0]*x[1] -2*x[2]**2 -5*x[5] + 11*x[6])
g.append(g4)

x0 = np.array([1., 2., 0., 4., 0., 1., 1.])


penalty_method(f, g, h, x0);

**********************************************************************
PM: STOP! KKT condition is verified.
PM: The optimum point is: [ 2.33050762  1.95137396 -0.47755362  4.36573318 -0.62448774  1.03813208
  1.59422536] with lambda= [1.139720432198601, 0.0, 0.0, 0.36863051533146063] and mu= []
PM: The optimum value of the bbjective function is: 680.6294968894514
